# Movimiento JetBot

Este documento cubre los aspectos básicos relativos al movimiento del JetBot.

### Importar clase Robot

Para interactuar con el JetBot y programar su funcionamiento, debemos importar la clase ``Robot``. Esta clase proporciona una interfaz de alto nivel para controlar los motores del robot. La clase ``Robot`` se encuentra en el paquete ``jetbot``

> Un *paquete* en Python puede interpretarse como un directorio con un conjunto de ficheros con utilidades que pueden utilizarse en el código del usuario una vez importadas. Los ficheros contenidos en un paquete se conocen como *módulos*

Para importar la clase ``Robot``, seleccionar la celda siguiente y pulsar ``ctrl + enter`` o el botón ``play``. Esto ejecutará el código de la celda.


In [ ]:
from jetbot import Robot

Una vez importada, debemos inicializar una *instancia* de la clase ``Robot``. El siguiente comando crea una variable de nombre ``robot`` a partir del constructor por defecto de la clase ``Robot``.

In [ ]:
robot = Robot()

### Control del robot

Podemos utilizar la variable "robot" creada anteriormente  para controlar al robot. Para ordenar al robot que gire en dirección antihoraria a un 30% de su velocidad máxima, podemos ejecutar el siguiente comando:

> OJO: El siguiente comando ordena al robot que se mueva. Asegurarse de que tiene espacio y no está en peligro de caidas.

In [ ]:
robot.left(speed=0.3)

Para detener al robot, se puede llamar al método ``stop``

In [ ]:
robot.stop()

Si quisieramos programar el robot de manera que gire un periodo limitado de tiempo, podríamos utilizar el paquete ``time`` de Python.

In [ ]:
import time

Este paquete contiene la función ``sleep``, que permite detener la ejecución de código durante un número de segundos definido por el usuario, antes de proseguir con el siguiente comando. A continuación, se demuestra como conseguir que el robot gire hacia la izquierda durante medio segundo.

In [ ]:
robot.left(0.3)
time.sleep(0.5)
robot.stop()

El JetBot debería girar hacia la izquierda y detenerse al medio segundo.

> En este caso, no se ha definido explicitamente el parámetro ``speed=`` en la invocación del método ``left``. Python permite definir parámetros de una función especificando su nombre o a partir del orden en que estos han sido definidos.

La clase ``Robot`` define otros métodos para controlar el JetBot, como ``right``, ``forward`` y ``backwards``. De esta manera podríamos implementar un comportamiento que hiciera que el JetBot avance a un 50% de velocidad durante 2s, gire hacia la derecha durante 3s y retroceda durante 2s. 

Para crear una nueva celda en el cuaderno actual podemos seleccionar la celda previa y pulsar ``b`` o el botón ``+`` de la parte superior. Una vez creada la celda, añade el código necesario para que el JetBot realice el movimiento definido anteriormente.

### Control individual de motores

En celdas anteriores se demostró como es posible controlar al JetBot con funciones como ``left`` y ``right``. A continuación, se mostrarán métodos alternativos para controlar la velocidad de cada motor de manera independiente.

La primera opción es invocar al método ``set_motors`` definido en la clase *Robot*. Para realizar un giro en arco hacia la izquierda durante un segundo podemos fijar el motor izquieerdo a un 30% de velocidad y el derecho a 60%.

In [ ]:
robot.set_motors(0.3, 0.6)
time.sleep(1.0)
robot.stop()

El jetbot debería realizar un giro en arco hacia la izquierda. Existe una manera alternativa de realizar el mismo movimiento.

La clase ``Robot`` tiene dos atributos llamados ``left_motor`` y ``right_motor`` que representa cada motor del Jetbot. Estos atributos son a su vez instancias de la clase ``Motor`` que exponen un atributo ``value``. Este atributo es un [traitlet](https://github.com/ipython/traitlets) que genera ``eventos`` cuando se les asigna un nuevo valor. En la práctica, esto permite ejecutar funciones predefinidas cada vez que el valor de este atributo cambia. En el caso de la clase ``Motor``, se le ha asignado una función que actualiza los comandos del motor cada vez que el valor de ``value`` cambia. 

De esta manera, para replicar el comportamiento de celdas anteriores, podemos ejecutar los siguientes comandos.

In [ ]:
robot.left_motor.value = 0.3
robot.right_motor.value = 0.6
time.sleep(1.0)
robot.left_motor.value = 0.0
robot.right_motor.value = 0.0

El JetBot debería replicar el movimiento de celdas anteriores.

### Asignar elementos interfaz a motores

Una funcionalidad interesante de los [traitlets](https://github.com/ipython/traitlets) es que es posible encadenar los eventos generados por los mismos. 

Esto nos permite definir componentes gráficos controlados desde el navegador (denominados ``widgets``) cuyo comportamiendo está asociado al traitlet de cada motor. De esta manera podemos visualizar el valor de cada uno de los traitlets y modificarlo en tiempo real.

Para mostrar esta funcionalidad crearemos una interfaz con dos sliders que utilizaremos para controlar los motores.


In [ ]:
import ipywidgets.widgets as widgets
from IPython.display import display

# crear dos sliders con rango [-1.0, 1.0]
left_slider = widgets.FloatSlider(description='left', min=-1.0, max=1.0, step=0.01, orientation='vertical')
right_slider = widgets.FloatSlider(description='right', min=-1.0, max=1.0, step=0.01, orientation='vertical')

# crear un contenedor para posicionar los sliders
slider_container = widgets.HBox([left_slider, right_slider])

# mostrar el contenedor en la celda
display(slider_container)

En la celda anterior de se debería mostrar los sliders verticales.

> HELPFUL TIP:  In Jupyter Lab, you can actually "pop" the output of cells into entirely separate window!  It will still be 
> connected to the notebook, but displayed separately.  This is helpful if we want to pin the output of code we executed elsewhere.
> To do this, right click the output of the cell and select ``Create New View for Output``.  You can then drag the new window
> to a location you find pleasing.

Si desplazamos los sliders comprobaremos que el valor de cada slider no se transmite el JetBot. Eso es porque aún no hemos asociado cada slider con el traitlet de cada motor. Para ello, utilizaremos la función ``link`` del paquete traitlets.

In [ ]:
import traitlets

left_link = traitlets.link((left_slider, 'value'), (robot.left_motor, 'value'))
right_link = traitlets.link((right_slider, 'value'), (robot.right_motor, 'value'))

Ahora podemos comprobar como el movimiento de los sliders se transmite al motor.

La función ``link`` que utilizamos anteriormente crea un link bidireccional. Esto significa que si cambiamos el valor de los motores de otro modo, los sliders también se actualizarán. Para comprobarlo, ejecuta el siguiente código:

In [ ]:
robot.forward(0.3)
time.sleep(1.0)
robot.stop()

Deberías observar como los sliders responden a los comandos del motor. Si quisieramos eliminar la conexión entre los sliders y los motores podemos llamar al método ``unlink`` en cada link.

In [ ]:
left_link.unlink()
right_link.unlink()

Es posible crear una conexión unidireccional, por ejemplo en el caso en que se pretenda utilizar los sliders para mostrar los valores de cada motor, sin capacidad para modificarlos. Para conseguirlo podemos utilizar la función ``dlink``. El primer parámetro de la función es el traitlet de origen y el segundo el objetivo.

In [ ]:
left_link = traitlets.dlink((robot.left_motor, 'value'), (left_slider, 'value'))
right_link = traitlets.dlink((robot.right_motor, 'value'), (right_slider, 'value'))

Si movemos los sliders podemos comprobar que el robot no responde. Por el contrario, si modificamos el valor de los motores mediante otro método, comprobaremos que los sliders se actualizan correctamente.

### Asignar funciones a eventos

Otro método de utilizar traitlets es el de asignar funciones (ej. ``forward``) a eventos. Estas funciones se invocarán cada vez que ocurra un cambio en un objeto, y se les transmitirá información sobre dicho cambio como el valor previo y el valor nuevo.

A continuación, crearemos una interfaz con botones que utilizaremos para controlar el robot.

In [ ]:
# crear botones
button_layout = widgets.Layout(width='100px', height='80px', align_self='center')
stop_button = widgets.Button(description='stop', button_style='danger', layout=button_layout)
forward_button = widgets.Button(description='forward', layout=button_layout)
backward_button = widgets.Button(description='backward', layout=button_layout)
left_button = widgets.Button(description='left', layout=button_layout)
right_button = widgets.Button(description='right', layout=button_layout)

# mostrar botones

middle_box = widgets.HBox([left_button, stop_button, right_button], layout=widgets.Layout(align_self='center'))
controls_box = widgets.VBox([forward_button, middle_box, backward_button])
display(controls_box)

Se debería mostrar un conjunto de controles para manejar el robot, aunque de momento no tienen funcionalidad asociada.

Para crear esta funcionalidad debemos implementar una serie de funciones que se asignarán al evento ``on_click`` de cada evento.

In [ ]:
def stop(change):
    robot.stop()
    
def step_forward(change):
    robot.forward(0.4)
    time.sleep(0.5)
    robot.stop()

def step_backward(change):
    robot.backward(0.4)
    time.sleep(0.5)
    robot.stop()

def step_left(change):
    robot.left(0.3)
    time.sleep(0.5)
    robot.stop()

def step_right(change):
    robot.right(0.3)
    time.sleep(0.5)
    robot.stop()

Una vez definidas las funciones, se asignarán a los eventos on_click de cada botón.

In [1]:
# asociar botones a acciones
stop_button.on_click(stop)
forward_button.on_click(step_forward)
backward_button.on_click(step_backward)
left_button.on_click(step_left)
right_button.on_click(step_right)

NameError: name 'stop_button' is not defined

Cada click de un botón debería producir un movimiento en el robot.

### Protocolo parada ante pérdida de conexión

A continuación, se muestra como definir un una función que detenga el robot en caso de que se pierda la conexión con el robot. Para ello, se crea una instancia de la clase ``Heartbeat`` que permite determinar si la conexión con el robot es estable de manera automática. 

Si la comunicación entre el navegador y el robot no se realiza correctamente entre dos heartbeats, el atributo ``status`` del heartbeat pasará a ``dead``. Tan pronto como se restaure la conexión, el atributo ``status`` volverá a ``alive``.

El periodo (en segundos) en que se evalúa el heartbeat puede ser fijado mediante el slider asociado.

In [ ]:
from jetbot import Heartbeat

heartbeat = Heartbeat()

# esta función será invocada cada vez que el estado del heartbeat cambie
def handle_heartbeat_status(change):
    if change['new'] == Heartbeat.Status.dead:
        robot.stop()
        
heartbeat.observe(handle_heartbeat_status, names='status')

period_slider = widgets.FloatSlider(description='period', min=0.001, max=0.5, step=0.01, value=0.5)
traitlets.dlink((period_slider, 'value'), (heartbeat, 'period'))

display(period_slider, heartbeat.pulseout)

Si ejecutamos el siguiente código para arrancar los motores y bajamos el slider de manera que la condición de heartbeat no pueda satisfacerse, comprobaremos como se detiene el motor. También podría desconectarse el robot o PC del robot.

In [ ]:
robot.left(0.2) 

# Reducir el valor del slider hasta que se detenga el robot